In [1]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras.preprocessing import image

np.random.seed(17)
tf.random.set_seed(17)
random.seed(17)

In [2]:
resnet = tf.keras.applications.DenseNet201()

In [3]:
!pip install pillow

In [4]:
cat_img = tf.keras.preprocessing.image.load_img('cat.jpg', target_size=(224, 224))

In [5]:
cat_img.show()

In [6]:
# далее необходимо представить картинку в виде массива

cat_img_array = image.img_to_array(cat_img)
cat_img_array = np.expand_dims(cat_img_array, axis=0)

In [7]:
# Загружаем предобученную модель DenseNet201
model = tf.keras.applications.DenseNet201(weights='imagenet')

In [8]:
# Предобрабатываем изображение
processed_img = tf.keras.applications.densenet.preprocess_input(cat_img_array)

In [9]:
pred = model.predict(processed_img)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [10]:
# Декодируем предсказания
decoded_pred = tf.keras.applications.densenet.decode_predictions(pred, top=1)[0]

In [11]:
decoded_pred

[('n02124075', 'Egyptian_cat', 0.16333994)]

In [13]:
# Замораживаем все слои
for layer in model.layers:
    layer.trainable = False

In [14]:
# Размораживаем последние 10 слоев
for layer in model.layers[-10:]:
    layer.trainable = True

In [16]:
# заменим активацию на последнем слое

model.layers[-1].activation = tf.keras.activations.relu

In [19]:
# Создаем новую модель для бинарной классификации
model_cats = tf.keras.models.Sequential([
   model,
   tf.keras.layers.Dense(1, activation='sigmoid')
]) 

In [37]:
# скомпилируем получившуюся модель, добавив необходимые метрики

accuracy = tf.keras.metrics.binary_accuracy
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

# как и в прошлый раз, F1 напишем сами
def f1_metrics(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2 * ((prec * rec) / (prec + rec + 1e-7))
    
model_cats.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
                   loss=tf.keras.losses.binary_crossentropy,
                   metrics=[accuracy, precision, recall, f1_metrics])

In [38]:
trainable_params = np.sum([tf.keras.backend.count_params(w) for w in model_cats.trainable_weights])
print(f"Количество тренируемых параметров: {trainable_params}")

Количество тренируемых параметров: 2204625


In [39]:
# функция предподготовки картинки для модели

def preprocess_image(file):
    img = tf.keras.preprocessing.image.load_img(file, target_size=(224, 224))  # загружаем в нужном разрешении
    img = tf.keras.preprocessing.image.img_to_array(img)  # конвертируем в массив
    img = tf.keras.applications.densenet.preprocess_input(img)  # препроцессинг для resnet
    return img

In [40]:
import os

In [41]:
# добавляем пары (картинка, 1) для картинок с котами
cats = [(preprocess_image('pics/cats/'+file), 1) for file in os.listdir('pics/cats')]

# и пары (картинка, 0) для картинок без котов
nocats = [(preprocess_image('pics/nocats/'+file), 0) for file in os.listdir('pics/nocats')]

In [42]:
# сливаем оба списка вместе

all_pics = cats + nocats

In [43]:
# и перемешиваем данные

random.shuffle(all_pics)

In [44]:
# в x отправляем картинки, а в y - прикреплённые к ним лейблы

x = np.array([a[0] for a in all_pics])
y = np.array([a[1] for a in all_pics])

In [45]:
# делим данные на трейн, валидацию и тест традиционным образом

def train_val_test_split(x, val_frac=0.15, test_frac=0.15):
    x_train = x[:round((1 - val_frac - test_frac) * len(x))]
    x_val = x[round((1 - val_frac - test_frac) * len(x)):round((1 - test_frac) * len(x))]
    x_test = x[round((1 - test_frac) * len(x)):]
    return x_train, x_val, x_test


x_train, x_val, x_test = train_val_test_split(x)
y_train, y_val, y_test = train_val_test_split(y)

In [46]:
# настроек у этого класса куда больше, но для примера возьмём только самые основные 

datagen = tf.keras.preprocessing.image.ImageDataGenerator()

In [47]:
# будем отслеживать обучение в Tensorboard

tb_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/tl_densenet_cats', histogram_freq=1)

# и уменьшать lr на плато

annealing = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1)

In [49]:
bs = 16  # размер батча

history = model_cats.fit(
    datagen.flow(x_train, y_train, batch_size=bs),
    validation_data=(x_val, y_val),
    steps_per_epoch=len(x_train)//bs,
    epochs=50,
    callbacks=[tb_callback, annealing]
)

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 28s 997ms/step - binary_accuracy: 0.4393 - f1_metrics: 0.3740 - loss: 1.0719 - precision_1: 0.3315 - recall_1: 0.5315 - val_binary_accuracy: 0.4706 - val_f1_metrics: 0.3602 - val_loss: 0.9026 - val_precision_1: 0.3226 - val_recall_1: 0.4000 - learning_rate: 1.0000e-05
Epoch 2/50
 1/20 ━━━━━━━━━━━━━━━━━━━━ 11s 598ms/step - binary_accuracy: 0.4375 - f1_metrics: 0.4706 - loss: 0.8668 - precision_1: 0.5000 - recall_1: 0.4444

/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 291ms/step - binary_accuracy: 0.4375 - f1_metrics: 0.4706 - loss: 0.8668 - precision_1: 0.5000 - recall_1: 0.4444 - val_binary_accuracy: 0.4706 - val_f1_metrics: 0.3602 - val_loss: 0.8928 - val_precision_1: 0.3226 - val_recall_1: 0.4000 - learning_rate: 1.0000e-05
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 17s 862ms/step - binary_accuracy: 0.5180 - f1_metrics: 0.3997 - loss: 0.8581 - precision_1: 0.3798 - recall_1: 0.4224 - val_binary_accuracy: 0.5294 - val_f1_metrics: 0.3921 - val_loss: 0.7453 - val_precision_1: 0.3704 - val_recall_1: 0.4000 - learning_rate: 1.0000e-05
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 292ms/step - binary_accuracy: 0.5000 - f1_metrics: 0.0000e+00 - loss: 0.9901 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_binary_accuracy: 0.5294 - val_f1_metrics: 0.3921 - val_loss: 0.7398 - val_precision_1: 0.3704 - val_recall_1: 0.4000 - learning_rate: 1.0000e-05
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 18s 896ms/step - binary_accuracy: 0.5906 -

In [50]:
model_cats.evaluate(x_test, y_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 688ms/step - binary_accuracy: 0.8841 - f1_metrics: 0.8639 - loss: 0.2372 - precision_1: 0.8846 - recall_1: 0.8214


[0.23724624514579773,
 0.8840579986572266,
 0.8846153616905212,
 0.8214285969734192,
 0.8639347553253174]